In [15]:
from myutils.DB import DB

mydb = DB('stocks')
mydb.tbl, mydb.wre = ('h_stockHistory_board',f"add1='SOXL'")
maxday = mydb.get("max(add0)",many=1,assoc=False)

mydb.wre = f"add1='SOXL' and add0='{maxday}'"
(clp,up,dn) = mydb.get('add3,add9,add10',many=1,assoc=False)

print(float(clp))
print(int(up)+1)
print(dn)

9.26
8
0
